In [1]:
# -*- coding: utf-8 -*-

## Map Generalization for Polygons using Autoencode-like strucutures
## Adatped based on Master Thesis of SERCAN CAKIR "ROAD NETWORK EXTRACTION USING CNN"
## Author: Yu Feng, yuzz.feng@gmail.com
## 1. Version Author: SERCAN CAKIR

## Changes:
## 1. Two conv layers were added before the first down convlusional layer
## 2. Output can be any size during the evaluation

import matplotlib
matplotlib.use('Agg') # necessary for linux kernal
import matplotlib.pyplot as plt

import os
import numpy as np
from numpy import random
np.random.seed(7)
import keras
from keras.models import Sequential
from keras.callbacks import History
from keras.layers.core import Dropout
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers import MaxPooling2D, Conv2DTranspose
from keras import backend as K

from osgeo import gdal
from sklearn.model_selection import train_test_split
from skimage.util.shape import view_as_windows

# function to read .tif image files
def readImg(img):
    # Read heatmap
    image = gdal.Open(img)
    # Get band of heatmap, it's gray scale image!
    img_band = image.GetRasterBand(1)
    # Read the image as array
    image = img_band.ReadAsArray()
    # Normalize the pixel values in the range 0-1 acc. to max. normalization
    image = (image - image.min()) / (image.max() - image.min())
    return image.astype('float32')

# function to create image patches
def imagePatches(img1, p_w, p_h, stride):
    img_1 = view_as_windows(img1, (p_w, p_h), stride)
    a, b, h, w = img_1.shape
    img_1_1 = np.reshape(img_1, (a * b, p_w, p_h))
    return img_1_1

# functions to remove fully black images from heatmap and target data, and all the correspondences
def removeBlackImg(img_patch):
    patch_list = []
    patch_list_new = []
    for i in range(len(img_patch)):
        patch_list.append(img_patch[i])
        if patch_list[i].max() != 0:
            patch_list_new.append(img_patch[i])
    return patch_list_new

# remove roads if heats are black
def removeCorrespondence(road, heat):  
    patch_road_list = []
    patch_heat_list = []
    patch_road_list_new = []
    for i in range(len(road)):
        patch_road_list.append(road[i])
        patch_heat_list.append(heat[i])
        if patch_heat_list[i].max() != 0:
            patch_road_list_new.append(road[i])
    return patch_road_list_new



/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# function to load a saved model
def LoadModel(model_json):
    from keras.models import model_from_json
    json_file = open(model_json)
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    return loaded_model


##### function to calculate evaluation parameters (F1-Score, Precision, Recall) ######
def evaluation(model, x_test, y_test, patch_size):
    precision = []
    recall = []
    f1Score = []
    import math
    for k in range(len(x_test_sim)):
        y_pred = model.predict(x_test_sim[k:k + 1])
        y_pred = np.reshape(y_pred, (32 * 32))

        y_true = y_test_sim[k:k + 1]
        y_true = np.reshape(y_true, (32 * 32))

        TP = 0
        FP = 0
        FN = 0
        TN = 0

        y_pred = np.round(y_pred)
        for i in range(len(y_pred)):
            if y_true[i] == y_pred[i] == 1:
                TP += 1
            elif y_pred[i] == y_true[i] == 0:
                TN += 1
            elif y_pred[i] == 1 and y_true[i] != y_pred[i]:
                FP += 1
            elif y_pred[i] == 0 and y_true[i] != y_pred[i]:
                FN += 1

        precision.append(TP / (TP + FP + K.epsilon()))  # completeness
        recall.append(TP / (TP + FN))  # correctness
        beta = 1
        f1Score.append((math.pow(beta, 2) + 1) * TP / ((math.pow(beta, 2) + 1) * TP + math.pow(beta, 2) * FN + FP))
        # eval_list = [precision,  recall, f1Score]

    avg_precision = sum(precision) / len(precision)
    avg_recall = sum(recall) / len(precision)
    avg_f1score = sum(f1Score) / len(precision)
    avg_eval_param = [avg_precision, avg_recall, avg_f1score]
    return avg_eval_param

In [2]:
##### Building the CNN archıtecture with "Sequential Model" 
##### (model looks like autoencoder)
def create_model(optimizer, input_shape):
    model = Sequential()
    
    droprate = 0.3

    model.add(Conv2D(filters=24, kernel_size=(5, 5),
              strides=(1, 1), padding='same',
              activation='relu', input_shape=input_shape, kernel_initializer='random_uniform',
              name="flat_conv_a"))
    model.add(Dropout(droprate))
    
    model.add(Conv2D(filters=24, kernel_size=(5, 5),
              strides=(1, 1), padding='same',
              activation='relu',name="flat_conv_b"))
    model.add(Dropout(droprate))
    
#    model.add(Conv2D(filters=24, kernel_size=(3, 3),
#              strides=(1, 1), padding='same',
#              activation='relu',name="flat_conv_c"))
#    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
#    model.add(Dropout(droprate))
    
    ## Encoding (down-sampling) ###   
    model.add(Conv2D(filters=24, kernel_size=(5, 5),
                     strides=(2, 2), padding='same',
                     activation='relu', #input_shape=input_shape, kernel_initializer='random_uniform',
                     name="down_conv_1"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='relu', name="flat_conv_1"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='relu', name="flat_conv_2"))
    model.add(Dropout(droprate))
    ##############################################################################
    
#    model.add(Conv2D(filters=24, kernel_size=(3, 3),
#              strides=(1, 1), padding='same',
#              activation='relu',name="down_conv_2"))
#    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
#    model.add(Dropout(droprate))
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     strides=(2, 2), padding='same',
                     activation='relu', name="down_conv_2"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=128, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='relu', name="flat_conv_3"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=256, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='relu', name="flat_conv_4"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=256, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='relu', name="flat_conv_5"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=256, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='relu', name="flat_conv_6"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=256, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='relu', name="flat_conv_7"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=128, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='relu', name="flat_conv_8"))
    model.add(Dropout(droprate))
    ###############################################################################
    model.add(UpSampling2D(size=(2, 2), name='up_samp_1'))
    
#    model.add(Conv2DTranspose(filters=64, kernel_size=(3, 3), strides=(2, 2), 
#                              padding='same', activation='softmax'))
    

    model.add(Conv2D(filters=64, kernel_size=(4, 4),
                     strides=(1, 1), padding='same',
                     activation='relu', name="up_conv_1"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='relu', name="flat_conv_9"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='relu', name="flat_conv_10"))
    model.add(Dropout(droprate))
    ###############################################################################
    model.add(UpSampling2D(size=(2, 2), name='up_samp_2'))
    
#    model.add(Conv2DTranspose(filters=64, kernel_size=(3, 3), strides=(2, 2), # Lead the accuracy to 0.78
#                              padding='same', activation='softmax'))

    model.add(Conv2D(filters=24, kernel_size=(4, 4),
                     strides=(1, 1), padding='same',
                     activation='relu', name="up_conv_2"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=12, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='relu', name="flat_conv_11"))
    model.add(Dropout(droprate))

    model.add(Conv2D(filters=1, kernel_size=(3, 3),
                     strides=(1, 1), padding='same',
                     activation='sigmoid', name="flat_conv_12"))
    # model.add(Activation(our_activation))
    model.add(Dropout(droprate))

    # Compile model with Adam optimizer and binary cross entropy loss function
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['acc'])
    return model


# model_ex1.summary()

##################################################################################################################################
#class Covariance(keras.callbacks.Callback):
#
#    def on_train_begin(self, logs={}):
#        self.avg_cov1 = []
#        self.avg_cov2 = []
#
#    def on_epoch_end(self, epoch, logs={}):
#        x_val1, y_true1 = x_train_sim, y_train_sim
#        y_pred1 = self.model.predict(x_val1)
#
#        x_val2, y_true2 = x_test_sim, y_test_sim
#        y_pred2 = self.model.predict(x_val2)
#
#        cov_1 = []
#        max_cov1 = []
#        for i in range(len(y_pred1)):
#            y_pred_clip1 = np.reshape(y_pred1[i][1:(len(y_pred1[1]) - 1), 1:(len(y_pred1[1]) - 1)],
#                                      ((len(y_pred1[1]) - 2) ** 2, 1))  # (len(y_pred1)-1)=31
#            y_true_1 = imagePatches(np.reshape(y_true1[i], (len(y_pred1[1]), len(y_pred1[1]))), (len(y_pred1[1]) - 2),
#                                    (len(y_pred1[1]) - 2), 1)
#            for k in range(0, 9):
#                cov_1.append(sum((y_pred_clip1 - y_pred_clip1.mean()) * (
#                            np.reshape(y_true_1[k], ((len(y_pred1[1]) - 2) ** 2, 1)) - (
#                        np.reshape(y_true_1[k], ((len(y_pred1[1]) - 2) ** 2, 1)).mean()))) / len(
#                    y_pred_clip1))  # 900
#        for j in range(0, 9 * len(y_pred1), 9):
#            max_cov1.append(max(cov_1[j:j + 9]))
#
#        cov_2 = []
#        max_cov2 = []
#        for i in range(len(y_pred2)):
#            y_pred_clip2 = np.reshape(y_pred2[i][1:(len(y_pred2[1]) - 1), 1:(len(y_pred2[1]) - 1)],
#                                      ((len(y_pred2[1]) - 2) ** 2, 1))  # (len(y_pred1)-1)=31
#            y_true_2 = imagePatches(np.reshape(y_true2[i], (len(y_pred2[1]), len(y_pred2[1]))), (len(y_pred2[1]) - 2),
#                                    (len(y_pred2[1]) - 2), 1)
#            for k in range(0, 9):
#                cov_2.append(sum((y_pred_clip2 - y_pred_clip2.mean()) * (
#                            np.reshape(y_true_2[k], ((len(y_pred2[1]) - 2) ** 2, 1)) - (
#                        np.reshape(y_true_2[k], ((len(y_pred2[1]) - 2) ** 2, 1)).mean()))) / len(
#                    y_pred_clip2))  # 900
#        for j in range(0, 9 * len(y_pred2), 9):
#            max_cov2.append(max(cov_2[j:j + 9]))
#        print('\n avg_cov: {} - val_avg_cov: {}\n'.format(np.mean(max_cov1), np.mean(max_cov2)))
#
#        self.avg_cov1.append(np.mean(max_cov1))
#        self.avg_cov2.append(np.mean(max_cov2))


##################################################################################################################################
class LearningRateTracker(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.lr_list = []

    def on_epoch_end(self, epoch, logs={}):
        optimizer = self.model.optimizer
        # lr = K.eval(optimizer.lr * (1. / (1. + optimizer.decay * optimizer.iterations)))
        lr = K.eval(
            optimizer.lr * (1. / (1. + optimizer.decay * K.cast(optimizer.iterations, K.dtype(optimizer.decay)))))
        print('\n LR: {}\n'.format(lr))
        self.lr_list.append(lr)

##################################################################################################################################
class SaveWeights(keras.callbacks.Callback):  # Saves weights after each 25 epochs
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 49 == 0:
            model_json = self.model.to_json()
            with open("model_" + str(epoch) + ".json", "w") as json_file:
                json_file.write(model_json)
            self.model.save_weights("weights_model_" + str(epoch) + ".h5")
            print("Saved model-weights to disk")

##################################################################################################################################

In [10]:
# Order the image dimension acc. to TensorFlow (batc_hsize, rows, cols, channels)
K.set_image_dim_ordering('tf')

# set the working directory
#os.chdir(r'F:\sercan\input_images')
PATH = os.getcwd()
#plt.gray()
#plt.show()


# load simulated heat map (TRAJECTORY SIMULATION) and target road for Hannover ####
sim_heatmap_hannover = readImg(r"input2.tif")
sim_road_hannover = readImg(r"output2.tif")

p_size_1 = 128
# hm: heatmap
#sim_hm_patches_32 = imagePatches(sim_heatmap_hannover, p_size_1, p_size_1, p_size_1)
#sim_road_patches_32 = imagePatches(sim_road_hannover, p_size_1, p_size_1, p_size_1)
#
#sim_road_patches_32_new = removeCorrespondence(sim_road_patches_32, sim_hm_patches_32)
#sim_hm_patches_32_new = removeCorrespondence(sim_hm_patches_32, sim_road_patches_32)
#sim_road_patches_32_new_new = removeBlackImg(sim_road_patches_32)

sim_hm_patches_overlap = imagePatches(sim_heatmap_hannover, p_size_1, p_size_1, int(p_size_1 / 2))
sim_road_patches_overlap = imagePatches(sim_road_hannover, p_size_1, p_size_1, int(p_size_1 / 2))
sim_road_patches_overlap_new = removeCorrespondence(sim_road_patches_overlap, sim_hm_patches_overlap)
sim_hm_patches_overlap_new = removeCorrespondence(sim_hm_patches_overlap, sim_road_patches_overlap)
sim_road_patches_overlap_new_new = removeBlackImg(sim_road_patches_overlap)


#sim_hm_patches_32_new = sim_hm_patches_32_new + sim_hm_patches_overlap_new
#sim_road_patches_32_new_new = sim_road_patches_32_new_new + sim_road_patches_overlap_new_new

sim_hm_patches_32_new = sim_hm_patches_overlap_new
sim_road_patches_32_new_new = sim_road_patches_overlap_new_new

print(len(sim_hm_patches_32_new))


### experience 1 - simulated hm
index_list_sim = list(range(len(sim_hm_patches_32_new)))
random.shuffle(index_list_sim)

idx_sim = 200
index_list_test_sim = index_list_sim[-idx_sim:]
index_list_test_sim.sort()
sim_hm_test = [sim_hm_patches_32_new[i] for i in index_list_test_sim]
sim_road_test = [sim_road_patches_32_new_new[i] for i in index_list_test_sim]

index_list_train_sim = index_list_sim[:-idx_sim]
index_list_train_sim.sort()
sim_hm_train = [sim_hm_patches_32_new[i] for i in index_list_train_sim]
sim_road_train = [sim_road_patches_32_new_new[i] for i in index_list_train_sim]

x_train_sim = np.reshape(sim_hm_train, (len(sim_hm_train), p_size_1, p_size_1, 1))
y_train_sim = np.reshape(sim_road_train, (len(sim_road_train), p_size_1, p_size_1, 1))
x_test_sim = np.reshape(sim_hm_test, (len(sim_hm_test), p_size_1, p_size_1, 1))
y_test_sim = np.reshape(sim_road_test, (len(sim_road_test), p_size_1, p_size_1, 1))

# save image patch arrays
np.save("x_train_sim.npy", x_train_sim)
np.save("y_train_sim.npy", y_train_sim)
np.save("x_test_sim.npy", x_test_sim)
np.save("y_test_sim.npy", y_test_sim)

# plt.imshow(np.reshape(x_test_sim[2], (p_size_1,p_size_1)))
# plt.imshow(np.reshape(y_test_sim[2], (p_size_1,p_size_1)))

input_shape1 = (None, None, 1) #x_train_sim[0].shape
print('Input Shape of the models', x_train_sim.shape)

opt1 = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
model_ex1 = create_model(opt1, input_shape1)

1485
Input Shape of the models (1285, 128, 128, 1)


In [11]:
##### Train the model
#covariance1 = Covariance()
History1 = History()
hist1 = model_ex1.fit(x_train_sim, y_train_sim,
                      batch_size=16,
                      epochs=500,
                      verbose=1,
                      shuffle=True,
                      callbacks=[History1],
                      validation_data=(x_test_sim, y_test_sim))

### Save history
History1_loss = History1.history['loss']
History1_acc = History1.history['acc']
History1_val_loss = History1.history['val_loss']
History1_val_acc = History1.history['val_acc']

thefile1 = open('History1_loss.txt', 'w')
for item in History1_loss:
    thefile1.write("%s\n" % item)

thefile2 = open('History1_acc.txt', 'w')
for item in History1_acc:
    thefile2.write("%s\n" % item)

thefile3 = open('History1_val_loss.txt', 'w')
for item in History1_val_loss:
    thefile3.write("%s\n" % item)

thefile4 = open('History1_val_acc.txt', 'w')
for item in History1_val_acc:
    thefile4.write("%s\n" % item)

### Save model
model_json1 = model_ex1.to_json()
with open("model_ex1.json", "w") as json_file:
    json_file.write(model_json1)
model_ex1.save_weights("weights_model_ex1.h5")
print("Saved model to disk")

Train on 1285 samples, validate on 200 samples
Epoch 1/500
1285/1285 [==============================] - 9s 7ms/step - loss: 1.2064 - acc: 0.7744 - val_loss: 0.1876 - val_acc: 0.9291
Epoch 2/500
1285/1285 [==============================] - 7s 6ms/step - loss: 1.0325 - acc: 0.8978 - val_loss: 0.2359 - val_acc: 0.8652
Epoch 3/500
1285/1285 [==============================] - 7s 6ms/step - loss: 1.0176 - acc: 0.9032 - val_loss: 0.1570 - val_acc: 0.9431
Epoch 4/500
1285/1285 [==============================] - 7s 6ms/step - loss: 1.0079 - acc: 0.9050 - val_loss: 0.1728 - val_acc: 0.9347
Epoch 5/500
1285/1285 [==============================] - 7s 6ms/step - loss: 1.0079 - acc: 0.9052 - val_loss: 0.1783 - val_acc: 0.9181
Epoch 6/500
1285/1285 [==============================] - 7s 6ms/step - loss: 1.0052 - acc: 0.9058 - val_loss: 0.1466 - val_acc: 0.9407
Epoch 7/500
1285/1285 [==============================] - 7s 6ms/step - loss: 1.0026 - acc: 0.9066 - val_loss: 0.1356 - val_acc: 0.9444
Epoch 8/

Epoch 61/500
1285/1285 [==============================] - 7s 6ms/step - loss: 1.0125 - acc: 0.9065 - val_loss: 0.1140 - val_acc: 0.9596
Epoch 62/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9845 - acc: 0.9118 - val_loss: 0.1120 - val_acc: 0.9584
Epoch 63/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9832 - acc: 0.9119 - val_loss: 0.1020 - val_acc: 0.9600
Epoch 64/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9813 - acc: 0.9120 - val_loss: 0.1057 - val_acc: 0.9608
Epoch 65/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9825 - acc: 0.9119 - val_loss: 0.0935 - val_acc: 0.9633
Epoch 66/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9830 - acc: 0.9119 - val_loss: 0.0951 - val_acc: 0.9612
Epoch 67/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9816 - acc: 0.9120 - val_loss: 0.0870 - val_acc: 0.9629
Epoch 68/500
1285/1285 [========================

1285/1285 [==============================] - 7s 6ms/step - loss: 0.9781 - acc: 0.9128 - val_loss: 0.0877 - val_acc: 0.9638
Epoch 122/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9783 - acc: 0.9128 - val_loss: 0.0958 - val_acc: 0.9618
Epoch 123/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9779 - acc: 0.9128 - val_loss: 0.0911 - val_acc: 0.9634
Epoch 124/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9774 - acc: 0.9128 - val_loss: 0.0921 - val_acc: 0.9626
Epoch 125/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9781 - acc: 0.9128 - val_loss: 0.0843 - val_acc: 0.9639
Epoch 126/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9772 - acc: 0.9129 - val_loss: 0.0863 - val_acc: 0.9646
Epoch 127/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9778 - acc: 0.9128 - val_loss: 0.0809 - val_acc: 0.9645
Epoch 128/500
1285/1285 [==============================

Epoch 181/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9751 - acc: 0.9136 - val_loss: 0.0792 - val_acc: 0.9652
Epoch 182/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9752 - acc: 0.9136 - val_loss: 0.0844 - val_acc: 0.9645
Epoch 183/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9746 - acc: 0.9136 - val_loss: 0.0811 - val_acc: 0.9646
Epoch 184/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9743 - acc: 0.9137 - val_loss: 0.0777 - val_acc: 0.9654
Epoch 185/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9731 - acc: 0.9138 - val_loss: 0.0906 - val_acc: 0.9646
Epoch 186/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9754 - acc: 0.9136 - val_loss: 0.0808 - val_acc: 0.9649
Epoch 187/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9747 - acc: 0.9137 - val_loss: 0.0870 - val_acc: 0.9640
Epoch 188/500
1285/1285 [================

Epoch 241/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9710 - acc: 0.9144 - val_loss: 0.0790 - val_acc: 0.9653
Epoch 242/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9718 - acc: 0.9144 - val_loss: 0.0734 - val_acc: 0.9657
Epoch 243/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9717 - acc: 0.9145 - val_loss: 0.0698 - val_acc: 0.9665
Epoch 244/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9707 - acc: 0.9145 - val_loss: 0.0732 - val_acc: 0.9663
Epoch 245/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9719 - acc: 0.9144 - val_loss: 0.0740 - val_acc: 0.9662
Epoch 246/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9705 - acc: 0.9145 - val_loss: 0.0714 - val_acc: 0.9663
Epoch 247/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9709 - acc: 0.9145 - val_loss: 0.0748 - val_acc: 0.9654
Epoch 248/500
1285/1285 [================

Epoch 301/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9688 - acc: 0.9150 - val_loss: 0.0674 - val_acc: 0.9669
Epoch 302/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9688 - acc: 0.9150 - val_loss: 0.0704 - val_acc: 0.9670
Epoch 303/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9688 - acc: 0.9151 - val_loss: 0.0709 - val_acc: 0.9665
Epoch 304/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9683 - acc: 0.9151 - val_loss: 0.0651 - val_acc: 0.9669
Epoch 305/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9686 - acc: 0.9150 - val_loss: 0.0680 - val_acc: 0.9668
Epoch 306/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9686 - acc: 0.9151 - val_loss: 0.0729 - val_acc: 0.9665
Epoch 307/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9683 - acc: 0.9151 - val_loss: 0.0649 - val_acc: 0.9670
Epoch 308/500
1285/1285 [================

Epoch 361/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9678 - acc: 0.9154 - val_loss: 0.0679 - val_acc: 0.9677
Epoch 362/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9670 - acc: 0.9155 - val_loss: 0.0688 - val_acc: 0.9672
Epoch 363/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9647 - acc: 0.9157 - val_loss: 0.0667 - val_acc: 0.9668
Epoch 364/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9660 - acc: 0.9156 - val_loss: 0.0668 - val_acc: 0.9670
Epoch 365/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9669 - acc: 0.9155 - val_loss: 0.0694 - val_acc: 0.9671
Epoch 366/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9668 - acc: 0.9155 - val_loss: 0.0626 - val_acc: 0.9678
Epoch 367/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9665 - acc: 0.9156 - val_loss: 0.0615 - val_acc: 0.9677
Epoch 368/500
1285/1285 [================

Epoch 421/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9645 - acc: 0.9159 - val_loss: 0.0605 - val_acc: 0.9683
Epoch 422/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9660 - acc: 0.9158 - val_loss: 0.0592 - val_acc: 0.9681
Epoch 423/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9653 - acc: 0.9158 - val_loss: 0.0629 - val_acc: 0.9682
Epoch 424/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9653 - acc: 0.9159 - val_loss: 0.0630 - val_acc: 0.9680
Epoch 425/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9663 - acc: 0.9158 - val_loss: 0.0614 - val_acc: 0.9681
Epoch 426/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9653 - acc: 0.9159 - val_loss: 0.0622 - val_acc: 0.9681
Epoch 427/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9662 - acc: 0.9158 - val_loss: 0.0617 - val_acc: 0.9681
Epoch 428/500
1285/1285 [================

Epoch 481/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9643 - acc: 0.9161 - val_loss: 0.0589 - val_acc: 0.9686
Epoch 482/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9644 - acc: 0.9161 - val_loss: 0.0599 - val_acc: 0.9685
Epoch 483/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9646 - acc: 0.9161 - val_loss: 0.0625 - val_acc: 0.9686
Epoch 484/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9646 - acc: 0.9161 - val_loss: 0.0570 - val_acc: 0.9686
Epoch 485/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9637 - acc: 0.9161 - val_loss: 0.0617 - val_acc: 0.9685
Epoch 486/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9632 - acc: 0.9162 - val_loss: 0.0631 - val_acc: 0.9684
Epoch 487/500
1285/1285 [==============================] - 7s 6ms/step - loss: 0.9657 - acc: 0.9160 - val_loss: 0.0625 - val_acc: 0.9683
Epoch 488/500
1285/1285 [================

In [30]:
image_arr = readImg(r"testexampleinput2.tif")
print(image_arr.shape)

conc2 = np.reshape(model_ex1.predict(np.reshape(image_arr, (1, image_arr.shape[0], image_arr.shape[1], 1))), 
                   (image_arr.shape[0], image_arr.shape[1] + 1))

fig = plt.figure(figsize=(image_arr.shape[1] / 1000, image_arr.shape[0] / 1000), dpi=100, frameon=False)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
plt.imshow(conc2)
#ax.imshow(conc2, aspect='normal')
fig.savefig("OutputTest", dpi=1000)

(1556, 2083)


In [31]:
### Plot history of average covariance - accuracy and loss of the models
plt.figure()
plt.plot(History1.history['loss'])
plt.plot(History1.history['val_loss'])
plt.title('loss & val_loss')
plt.legend(['train', 'test'], loc='upper right')
plt.savefig("loss", dpi=1000)

plt.figure()
plt.plot(History1.history['acc'])
plt.plot(History1.history['val_acc'])
plt.title('acc & val_acc')
plt.legend(['train', 'test'], loc='upper right')
plt.savefig("acc", dpi=1000)